# OpenAI API Examples

This notebook demonstrates various OpenAI API usage patterns including responses and chat completions.


## Setup


In [ ]:
#!pip install openai pydantic


In [ ]:
import os
from openai import OpenAI
from pydantic import BaseModel
from typing import List

# Initialize client
client = OpenAI(api_key='your_openai_api_key_here')


## Using client.responses.create


### Simple Client with Non-Reasoning Model


In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input="What is machine learning?"
)

print(response.output_text)


### Client with Reasoning Model


In [ ]:
response = client.responses.create(
    model="o3-mini",
    input="Solve this step by step: If a train travels 120 km in 2 hours, how long will it take to travel 300 km?"
)

print(response.output_text)


### Image as Input


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    }
                }
            ]
        }
    ]
)

print(response.choices[0].message.content)

### Temperature, Model, Instructions (Non-Reasoning)


In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input="Write a creative story about AI",
    instructions="You are a creative writing assistant. Write in a poetic style.",
    temperature=0.9
)

print(response.output_text)


### Responses with Reasoning Effort


In [ ]:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?",
)

print(response.output_text)

### Alternative Response Formats

In [ ]:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.output_text)

### Verbosity Parameter


In [ ]:
response = client.responses.create(
    model="gpt-5.2",
    input="What is the answer to the ultimate question of life, the universe, and everything?",
    text={
        "verbosity": "low"
    }
)

print(response.output_text)

In [ ]:
response = client.responses.create(
    model="gpt-5.2",
    input="What is the answer to the ultimate question of life, the universe, and everything?",
    text={
        "verbosity": "high"
    }
)

print(response.output_text)

### Multiple Turns


In [ ]:
response1 = client.responses.create(
    model="gpt-4o-mini",
    input="My name is Alice"
)

print("Turn 1:", response1.output_text)

response2 = client.responses.create(
    model="gpt-4o",
    input="What's my name?",
    previous_response_id=response1.id
)

print("\nTurn 2:", response2.output_text)


### Roles (Developer, User, Assistant)


In [ ]:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.output_text)


## Using client.chat.completions.create


### Basic Messages Array


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain neural networks in one sentence."}
    ]
)

print(response.choices[0].message.content)


### Structured Output with Pydantic


In [ ]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

## Research Paper Extraction 

In [ ]:
from pydantic import BaseModel

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
        {"role": "user", "content": "..."}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message.parsed

In [ ]:
research_paper 


## Moderation

In [ ]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel

class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "How do I prepare for a job interview?"}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed

In [ ]:
compliance

In [ ]:
completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "I want to hurt myself."}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed

In [ ]:
compliance